In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
import Pkg; Pkg.add("RobotZoo")
using RobotZoo: YakPlane
import RobotDynamics
using RobotDynamics: state_dim, control_dim
using ForwardDiff
using StaticArrays
using LinearAlgebra
using Rotations
using Printf
using Test
using TrajOptPlots
using JLD2
const RD = RobotDynamics
include("utils.jl")
include("airplane.jl")
const isautograder = @isdefined autograder;
const resfile = joinpath(@__DIR__,"q1.jld2")

  Activating environment at `~/Desktop/OptimalC/Homeworks/homework4-afariach/Project.toml`


    Updating registry at `~/.julia/registries/General`


   Resolving package versions...


  No Changes to `~/Desktop/OptimalC/Homeworks/homework4-afariach/Project.toml`
  No Changes to `~/Desktop/OptimalC/Homeworks/homework4-afariach/Manifest.toml`


"/Users/Augusto/Desktop/OptimalC/Homeworks/homework4-afariach/src/q1.jld2"

# Q1: iLQR with Quaternions (35 pts)
In this problem you'll adapt the iLQR algorithm you wrote in the last homework to work with 3D rotations, using the methods we talked about in class. We'll also see one of the drawbacks of DDP-based methods like iLQR: the need for good initialization. 

In this problem we'll be designing an aerobatic maneuver for a small-scale model airplane. The dynamics you'll be using use data extracted from real-world wind-tunnel testing. Since the dynamics are fairly complex, we'll skip describing them in detail. Instead we'll simply describe the state and control inputs:

$$ x = \begin{bmatrix}
    p \\ q \\ v \\ \omega
\end{bmatrix}, \;\; 
u = \begin{bmatrix}  
    u_\text{thr} \\ u_\text{ail} \\ u_\text{elev} \\ u_\text{rud}
\end{bmatrix}
$$
where $p \in \mathbb{R}^3$ is the position of the center of mass in the world frame, $q \in \mathbb{S}^3$ is the unit quaternion mapping vectors in the body frame to the world frame, $v \in \mathbb{R}^3$ is the linear velocity in the world frame, and $\omega \in \mathbb{R}^3$ is the angular velocity in the body frame. The controls are all scalar values ranging from 0-255 for the thrust, aileron, elevator, and rudder control surfaces.

If you want to look at the system dynamics, check out [this file](https://github.com/RoboticExplorationLab/RobotZoo.jl/blob/master/src/yak_plane.jl).


In [2]:
# Dynamics API
let
    model = YakPlane(UnitQuaternion)
    x,_ = rand(model)  # generate a random state and control input (with the orientation sampled correctly)
    u = Vector{Float64}(rand(0:255, 4))
    t  = 0.0  # time
    dt = 0.1  # time step
    
    # continuous dynamics
    dynamics(model, x, u)
    
    # discrete dynamics
    xnext = discrete_dynamics(model, x, u, t, dt);
    A,B = discrete_jacobian(model::YakPlane, x, u, t, dt)
    
end;  # wrap in let/end to avoid polluting the global state

## Part (a): Finding the trim conditions (7 pts)
To get a good answer out of shooting methods like iLQR that rely on a dynamically feasible initial guess, it's often a good idea to find some sort of "stable" initial trajectory. For this problem, we'll simply find the "trim" conditions for level flight, or the controls need to maintain level flight with zero acceleration. In principle, this is going to be very similar to the problem in the HW1 where we found the equilibrium state for the quadruped, except that here we're only constraining the accelerations (not velocities) to be zero.

To find the trim conditions, we'll solve the following optimization problem using Gauss-Newton:
$$
\begin{align}
& \text{minimize}_{u} && \frac{1}{2} (u - u_\text{guess})^T (u - u_\text{guess}) \\
& \text{subject to} && f_\text{accel}(x_\text{trim},u) = 0
\end{align}
$$
where $u_\text{guess}$ in an initial guess for the controls, $x_\text{trim}$ is the state vector (both position and velocity) we want to design our trim controls for. In our case, this will be level flight with a forward velocity along the x-axis of 5 m/s. $f_\text{accel}$ is a function that returns the linear and angular accelerations. You can just pull these out of a call to the continuous-time dynamics function.

In [3]:
# TASK: implement get_trim to find the trim controls using Newton's method (7 pts)
"""
    get_trim(model::YakPlane, x_trim, u_guess)

Calculate the trim controls for the YakPlane model, given the position and velocity specified by `x_trim`.
Find a vector of 4 controls close to those in `u_guess` that minimize the accelerations on the plane.
"""
function get_trim(model::YakPlane, x_trim, u_guess;
        verbose = false,
        tol = 1e-4,
        iters = 100)
        #Code from HW1 
        res_hist = Float64[]
        # n,m = size(u_guess)
        λ = zeros(6)
        ρ = 1e-3
        u = copy(u_guess) 
        # TODO: Finish the function
        for k = 1:iters
            B = ForwardDiff.jacobian(u_->dynamics(model, x_trim, u_), u)
            H = [I B[8:13,:]';B[8:13,:] zeros(6,6)]
            H = H + Diagonal([ones(4); -ones(6)])*ρ
            c = [(u-u_guess) + B[8:13,:]'*λ;dynamics(model,x_trim,u)[8:13]] 
            # kkt_conditions(model,x,u, λ, A, B) #
            Δz = H\c#[(x-x_guess) + A'*λ;2*(0.5e-3)*u + B'*λ;dynamics(model,x,u)] 
            Δu = Δz[1:4]
            Δλ = Δz[5:10]
            u = u-Δu
            λ = λ-Δλ
            res_hist = [res_hist,norm(Δz)]
            if norm(c) <= tol 
                break
            end 
        end 
    

    # TODO: Use Newton's method to find the trim controls
    #  TIP: Look at your solution the quadruped problem on HW1
    #  TIP: You should have 4 decision variables and 6 constraints
    #  TIP: You don't need to do anything special due to the rotation, just do normal (Gauss) Newton
    
    utrim = copy(u)

    # Return the trim controls
    return utrim
end

get_trim

In [4]:
# Test that the method finds zero acceleration
@testset "Part a" begin                                                           # POINTS = 7
    model = YakPlane(UnitQuaternion)
    x0 = SA[
        0,0,0, 
        -6.114599630220157e-8, 0.997155939027903, 0.0, 0.07536599539167085,
        5,0,0,
        0,0,0
    ]
    u_guess = fill(124.0,4)

    utrim = get_trim(model, x0, u_guess)
    
    # Make sure the accelerations are small
    @test norm(dynamics(model, x0, utrim)[8:13]) < 1e-3                           # POINTS = 5
    @test utrim ≈ load(resfile, "utrim") rtol=1e-3                                # POINTS = 2
end; # wrap in begin/end to avoid polluting the global state with variables

Test Summary: | Pass  Total
Part a        |    2      2


## Part (b): Computing the Error State (5 pts)
As we learned in class, we will compute the backward pass of iLQR on the error state, which for a single rigid body like the airplane will be an element of $\mathbb{R}^{12}$. We need a method for converting between our state vector in $\mathbb{R}^{9} \times \mathbb{S}^3$ into an error state. 

Before computing the error state, we need a few methods to work with quaternions. We'll need a way to multiply two quaternions. The easiest way to do this is by defining the linear operator $L(q_1) q_2$. Define the `lmult` method below.

In [5]:
# TASK: implement lmult (1 pt)

"""
    lmult(q)

Return the 4x4 matrix `L` such that `L(q1)*q2` is equivalent to the quaternion multiplication `q1*q2`,
where `q1` and `q2` follow the Hamilton convention (i.e. q[1] is the scalar part and q[2:4] is the vector part).
"""
function lmult(q)
    # TODO: calculate L(q)
    a = q[1]
    b = q[2]
    c = q[3]
    d = q[4]
    Lq = [a -b -c -d; b a -d c;c d a -b; d -c b a]
    return Lq
end

lmult

There are many ways to represent a three-parameter error quaternion, but we'll use the *Cayley Map*, which we've found to work best and is very computationally efficient. This is equivalent to converting the quaternion difference to a Rodrigues Parameter:
$$ g = \frac{1}{q_s} \begin{bmatrix} q_x \\ q_y \\ q_z \end{bmatrix} $$

Using the Cayley map, compute the error state, given two states $x$ and $x_0$. This is equivalent to $x - x_0$ for vector (Euclidean) states. Remember that the difference between two quaternions is just $q_0^\dagger q$ which is the same as $L(q_0^\dagger) q = L(q_0)^T q$.

In [6]:
# TODO: Implement the state_error function (2 pts)
"""
    state_parts(model::YakPlane)
Give a tuple of indices, `ip,iq,iv,iw` for extracting pieces of the state vector:
* `x[ip]` returns the 3 positions in the world frame
* `x[iq]` returns the 4 elements of the unit quaternion
* `x[iv]` returns the 3 linear velocities in the world frame
* `x[iw]` returns the 3 angular velocities in the body frame
"""
function state_parts(model::YakPlane)
    ip = SA[1,2,3]
    iq = SA[4,5,6,7]
    iv = SA[8,9,10]
    iw = SA[11,12,13]
    return ip, iq, iv, iw
end



function state_error(model::YakPlane, x, x0)
    # TODO: Compute the error state between x and x0
    p,q,v,w = state_parts(model)
    # @show p,q,v,w
    dp = x[p]-x0[p]
    qsub= lmult(x0[q])'*x[q]
    dq = (1/qsub[1])*(qsub[2:4])
    dv = x[v]-x0[v]
    dw = x[w]-x0[w]
    dx = [dp;dq;dv;dw]
    return dx
end


state_error (generic function with 1 method)

Now that we have a function to compute the error state, we're also going to need it's Jacobian (no surprise there). Remember that the *attitude Jacobian* is the Jacobian of the quaternion difference, and is given as:
$$
G(q) = \begin{bmatrix}
    -q_v^T \\ q_s I_3 + \hat{q}_v
\end{bmatrix}
$$
where $q_s \in \mathbb{R}$ and $q_v \in \mathbb{R}^{3}$ are the scalar and vector part of the quaternion, respectively, and $\hat{x}$ is the skew-symmetric matrix operator:
$$
\hat{x} = \begin{bmatrix}
 0   & -x_3 & x_2 \\
 x_3 &  0   &-x_1 \\
-x_2 &  x_1 & 0   \\
\end{bmatrix}
$$

In [7]:
# TODO: Implement the following methods
#       1. attitude_jacobian     (1 pt)
#       2. state_error_jacobian  (1 pt)

"""
    attitude_jacobian(q)

Return the 4x3 attitude Jacobian for the unit quaternion q following the Hamilton convention.
The attitude Jacobian is the Jacobian of `q*ϕ(dq)` with respect to `dq`,
where `ϕ` transforms the infinitessimal rotation `dq` to a unit quaternion.
"""
function attitude_jacobian(q)
    qs = q[1]
    qv = q[2:4]
    q_skew = [0 -qv[3] qv[2];qv[3] 0 -qv[1];-qv[2] qv[1] 0]
    G = [-qv';qs*I+q_skew]
    return G
end

"""
    state_error_jacobian(model, x)

Return the Jacobian of the `state_error` function, with respect to the state `x`.
"""
function state_error_jacobian(model, x)
    jac = zeros(length(x),length(x)-1)
    jac[1:3,1:3] = Matrix(I,3,3)
    jac[4:7,4:6] = attitude_jacobian(x[4:7])
    jac[8:10,7:9]= Matrix(I,3,3)
    jac[11:13,10:12]= Matrix(I,3,3)
    return jac 
end

state_error_jacobian

In [8]:
# Test the attitude and state error Jacobians
@testset "Part b" begin                                           # POINTS = 5
    model = YakPlane(UnitQuaternion)
    q1 = [0.15923376998393293, 0.26188238416859005, 0.944750125601984, 0.11623004574512731]
    q2 = [-0.511916585919045,  0.03735280673079172, -0.036349007931561726, -0.8574525797457913]
    x1 = [0.989607214000322, 0.51362326582473, 0.3616831284706765, -0.6336197238671922, 0.01573491125215133, -0.7172154648116752, -0.28962119247272433, 0.008365202831213558, 0.5515476660019731, 0.19996988824486706, 0.25027308097388, 0.8555278096133965, 0.9134644393324214]
    x2 = [0.45493888030931307, 0.8214867872560794, 0.587845774939719, 0.0859756782429905, -0.357328766626224, -0.4039767922209037, -0.8376915223630642, 0.274895624769145, 0.40424715886322726, 0.7686052831243393, 0.1958635140022562, 0.33959528953302853, 0.06666659269188946]
    u = [124.17276299136248, 135.60429047063715, 145.9949664404939, 24.18838184023805]
    
    @test lmult(q2)*q1 ≈ load(resfile, "q2q1") rtol=1e-6          # POINTS = 0.5
    @test lmult(q1)'q1 ≈ [1,0,0,0] rtol=1e-8                      # POINTS = 0.5
    
    dx = state_error(model, x1, x2)
    @test dx ≈ load(resfile, "dx") rtol=1e-6                      # POINTS = 2

    G0 = attitude_jacobian(q1)
    @test G0 ≈ load(resfile, "G1")                                # POINTS = 1
    G = state_error_jacobian(model, x1)
    @test G*dx ≈ load(resfile, "Gdx") rtol=1e-6                   # POINTS = 1
end;

Test Summary: | Pass  Total
Part b        |    5      5


## Part (c): A Better Cost Function (6 pts)
In our trajectory optimization problem, we're going to want to penalize distance from a reference trajectory or a goal, so we need a cost function that does this efficiently and correctly for our state vector, which includes quaternions. While we could use the error we computed above, this is very nonlinear and can impede convergence. Instead, we use the simple *geodesic distance* between two quaternions:
$$ \text{dist}_\text{geo}(q_1,q_2) = \min(1 \pm q_1^T q_2) $$

For our trajectory optimization problem we're going to use a cost function of the following form:
$$ \frac{1}{2}\left( (p-p_d)^T Q_p (p-p_d) + (v-v_d)^T Q_v (v-v_d) + (\omega - \omega_d)^T Q_\omega (\omega - \omega_d) \right)  + w \cdot \text{dist}_\text{geo}(q, q_d)$$
Use this to flush out the cost function below, and it's derivatives.

**NOTE**: You will be deducted points if you use FiniteDiff or ForwardDiff (or related packages) to compute the derivatives.

In [9]:
# TASK: Complete the following methods
#       1. stagecost (1 pt)
#       2. termcost  (2 pts)
#       3. gradient  (2 pts)
#       4. hessian   (1 pt)

"""
    RigidBodyCost{m,T}

A cost function that penalizes distance of a rigid body state specified by its 
position `p`, attitude `q` (a unit quaternion), linear velocity `v` and angular velocity `ω`.

The vector state use a standard quadratic penalty, and the attitude uses the geodesic distance, weighted by a scalar `w`.
Uses a quadratic penalty on the distance of the `m`-dimensional control vector from a reference control `u_ref`.
"""
struct RigidBodyCost{m,T}
    Qp::Diagonal{T,SVector{3,T}}
    w::T
    Qv::Diagonal{T,SVector{3,T}}
    Qω::Diagonal{T,SVector{3,T}}
    R::Diagonal{T,SVector{m,T}}
    p_ref::SVector{3,T}
    q_ref::SVector{4,T}
    v_ref::SVector{3,T}
    ω_ref::SVector{3,T}
    u_ref::SVector{m,T}
end

"""
    stagecost(cost::RigidBodyCost, x, u)

Calculate the cost for a single knot point for the `RigidBodyCost`. 
TIP: Call `termcost` and add the control cost, to avoid repeated code
"""
function stagecost(cost::RigidBodyCost, x, u)
    # TODO: Calculuate the stage cost
    ucost = (u-cost.u_ref)'*cost.R*(u-cost.u_ref)
    J = termcost(cost,x)+0.5*ucost
    return J
end

"""
    termcost(cost::RigidBodyCost, x)

Calculate the state-only terminal cost for a `RigidBodyCost`. 
Should use the geodesic distance to compute the error between the 
quaternion in `x` and `cost.q_ref`.
"""
function termcost(cost::RigidBodyCost, x)
    # TODO: Calculate the state-only cost
    p = x[1:3]
    q = x[4:7]
    v = x[8:10]
    w = x[11:13]
    pcost = (p-cost.p_ref)'*cost.Qp*(p-cost.p_ref)
    vcost = (v-cost.v_ref)'*cost.Qv*(v-cost.v_ref)
    wcost = (w-cost.ω_ref)'*cost.Qω*(w-cost.ω_ref)
    qcost = cost.w*min(1+q'*cost.q_ref,1-q'*cost.q_ref)
    J = 0.5*(pcost+vcost+wcost)+qcost
    return J
end

"""
    gradient(cost::RigidBodyCost, x, u)

Calculate the gradient of the cost function with respect to
the states `x` and controls `u`. Should return both as a tuple.
"""
function gradient(cost::RigidBodyCost, x, u)
    # TODO: Calculate the gradients with respect to both x and u
    p = x[1:3]
    q = x[4:7]
    v = x[8:10]
    w = x[11:13]
    grad_pcost = cost.Qp*(p-cost.p_ref)
    grad_vcost = cost.Qv*(v-cost.v_ref)
    grad_wcost = cost.Qω*(w-cost.ω_ref)
    #q_ref'*q
    #How to get the derivative of the quaternions?
    if q'*cost.q_ref > 0 
        grad_qcost = -cost.w*cost.q_ref
    else
        grad_qcost = cost.w*cost.q_ref
    end 
    grad_x = [grad_pcost;grad_qcost;grad_vcost;grad_wcost]
    grad_u = cost.R*(u-cost.u_ref)
    
    return grad_x, grad_u
end

"""
    hessian(cost::RigidBodyCost, x, u)

Calculate the Hessian of the cost function with respect to both the 
states `x` and controls `u` for a `RigidBodyCost`. 
Should return a tuple of both Hessians
"""
function hessian(cost::RigidBodyCost, x, u)
    # TODO: Calculate Cost Hessian  
    Q = zeros(length(x), length(x))
    Q[1:3,1:3] = cost.Qp
    #Hessian of Quaternions? Treat it as a vector? 
    Q[4:7,4:7] .= 0
    Q[8:10,8:10] = cost.Qv
    Q[11:13,11:13] = cost.Qω
    R = cost.R
    return Q, R
end

"""
    cost(obj, X, U)

Calculate the cost along an entire trajectory using a collection
of cost function types (stored in the iteratable collection `obj`).
"""
function cost(obj, X, U)
    J = zero(eltype(X[1]))
    for k = 1:length(U)
        J += stagecost(obj[k], X[k], U[k])
    end
    J += termcost(obj[end], X[end])
    return J
end

cost

In [10]:
# Test cost and derivatives
@testset "Part c" begin                                                                      # POINTS = 6
    model = YakPlane(UnitQuaternion)
    x = SA[0.45493888030931307, 0.8214867872560794, 0.587845774939719, 0.0859756782429905, -0.357328766626224, -0.4039767922209037, -0.8376915223630642, 0.274895624769145, 0.40424715886322726, 0.7686052831243393, 0.1958635140022562, 0.33959528953302853, 0.06666659269188946]
    u = SA[124.17276299136248, 135.60429047063715, 145.9949664404939, 24.18838184023805]
    ip,iq,iv,iw = state_parts(model)
    costfun = RigidBodyCost(
        Diagonal(@SVector fill(0.1, 3)),
        10.0, 
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(0.1, 3)),
        Diagonal(@SVector fill(1e-3, 4)),
        SA[0,0,0.],
        SA[1,0,0,0.],
        SA[5,0,0.],
        SA[0,0,0.],
        u
    )
    
    J1 = stagecost(costfun, x, u)
    xflip = [x[ip]; -x[iq]; x[iv]; x[iw]]
    @test stagecost(costfun, xflip, u) ≈ J1                                                  # POINTS = 1
    @test J1 ≈ load(resfile, "J1") rtol=1e-5                                                 # POINTS = 1
    
    Jterm = termcost(costfun, x)
    @test Jterm ≈ load(resfile, "Jterm") atol=1e-8 rtol=1e-5                                 # POINTS = 1

    @test ForwardDiff.gradient(x->termcost(costfun,x), x) ≈ gradient(costfun, x, u)[1]       # POINTS = 1
    @test ForwardDiff.gradient(u->stagecost(costfun,x,u), u) ≈ gradient(costfun, x, u)[2]    # POINTS = 1
    @test ForwardDiff.hessian(x->termcost(costfun,x), x) ≈ hessian(costfun, x, u)[1]         # POINTS = 0.5
    @test ForwardDiff.hessian(u->stagecost(costfun,x,u), u) ≈ hessian(costfun, x, u)[2]      # POINTS = 0.5
end;

Test Summary: | Pass  Total
Part c        |    7      7


## Part (d): Adapt the iLQR Algorithm (12 pts)
Now that we have all the building blocks, modify the iLQR algorithm to work with the airplane dynamics. You'll need to make modifications in two places:
1. The backward pass: You'll need to "convert" the Jacobians to be on the error state, by multiplying by the error state Jacobian. You'll need to handle the gradients, Hessians (remember the extra term) and dynamics Jacobians all slightly differently.
2. The forward pass: when simulating the dynamics forward, you'll need to apply the control law on the error state.

In [11]:
"""
    iLQRProblem{n,m,L}

Describes a trajectory optimization problem with `n` states, `m` controls, and 
a model of type `L`. 

# Constructor
    Problem(model::L, obj::Vector{<:QuadraticCost{n,m}}, tf, x0, xf) where {n,m,L}

where `tf` is the final time, and `x0` is the initial state. 
"""
struct iLQRProblem{n,m,L,O}
    model::L
    obj::Vector{O}
    N::Int
    tf::Float64
    x0::MVector{n,Float64}
    times::Vector{Float64}
    function iLQRProblem(model::L, obj::Vector{O}, tf, x0) where {L,O}
        n,m = size(model)
        @assert length(x0) == n
        T = length(obj)
        times = range(0, tf, length=T)
        new{n,m,L,O}(model, obj, T, tf, x0, times)
    end
end
Base.size(prob::iLQRProblem{n,m}) where {n,m} = (n,m,prob.N)

In [12]:
# TASK: Modify the backward pass to calculate the control law on the error state (8 pts)
"""
    backwardpass!(prob, P, p, K, d, X, U)

Evaluate the iLQR backward pass at state and control trajectories `X` and `U`, 
storing the cost-to-go expansion in `P` and `p` and the gains in `K` and `d`.

Should return ΔJ, expected cost reduction.
"""
function backwardpass!(prob::iLQRProblem{n,m}, P, p, K, d, X, U; 
        β=1e-6
    ) where {n,m}
    N = prob.N
    ΔJ = 0.0
    model = prob.model

    err_jac = state_error_jacobian(model,X[N-1])
    err_jac_n = state_error_jacobian(model,X[N])

    iq = state_parts(prob.model)[2]
    Iq = Diagonal(SA[0,0,0,1,1,1, 0,0,0, 0,0,0])

    # Calculate gradient and Hessian at terminal state
    Q, = hessian(prob.obj[N], X[N], 0*U[1])
    q, = gradient(prob.obj[N], X[N], 0*U[1]) 

    # TODO: Calculate the terminal cost-to-go, accounting for the group structure of the 3D rotation
    p[N] = err_jac_n'*q
    #Gradient 
    P[N] = err_jac_n'*Q*err_jac_n - Iq*(q[iq]'*X[N][iq])

    #Backward Pass
    for k = (N-1):-1:1

        err_jac = state_error_jacobian(model,X[k])
        err_jac_n = state_error_jacobian(model,X[k+1])

        # Cost Expansion
        Q,R =  hessian(prob.obj[k], X[k], U[k])
        q,r = gradient(prob.obj[k], X[k], U[k])

        # Dynamics derivatives
        dt = prob.times[k+1] - prob.times[k]
        A,B = discrete_jacobian(model, X[k], U[k], prob.times[k], dt)

        A = err_jac_n'*A*err_jac
        B = err_jac_n'*B
        
        Q = err_jac'*Q*err_jac - Iq*(q[iq]'*X[k][iq])
        q = err_jac'*q
        # TODO: Account for the group structure of 3D rotations using the attitude Jacobian (via state_error_jacobian)
        #  TIP: A,B,Q, and q should all be modified (including their size)
        
    
        gx = q + A'*p[k+1]
        gu = r + B'*p[k+1]
    
        Gxx = Q + A'*P[k+1]*A
        Guu = R + B'*P[k+1]*B
        Gux = B'*P[k+1]*A
       
        
        # Regularization
        Gux_reg = SMatrix{m,n-1}(Gux)
        Guu_reg = SMatrix{m,m}(Guu + β*Diagonal(@SVector ones(m)))
        
        # Calculate Gains
        F = cholesky(Symmetric(Guu_reg))
        d[k] .= (F\gu)
        K[k] .= (F\Gux_reg)
    
        # Cost-to-go Recurrence
        p[k] .= gx - K[k]'*gu + K[k]'*Guu*d[k] - Gux'*d[k]
        P[k] .= Gxx + K[k]'*Guu*K[k] - Gux'*K[k] - K[k]'*Gux
        ΔJ += gu'*d[k]
   
    end
    return ΔJ
end

backwardpass!

In [13]:
# TASK: Modify the forward pass algorithm to apply the control policy on the error state (4 pts)
"""
    forwardpass!(prob, X, U, K, d, ΔJ, J)

Evaluate the iLQR forward pass at state and control trajectories `X` and `U`, using
the gains `K` and `d` to simulate the system forward. The new cost should be less than 
the current cost `J` together with the expected cost decrease `ΔJ`.

Should return the new cost `Jn` and the step length `α`.
"""
function forwardpass!(prob::iLQRProblem{n,m}, X, U, K, d, ΔJ, J,
        Xbar = deepcopy(X), Ubar = deepcopy(U);
        max_iters=10,
    ) where {n,m}
    N = prob.N
    model = prob.model

    Jn = J
    # Line Search
    Xbar[1] = X[1]
    α = 1.0
    Jn = Inf
    
    for i = 1:max_iters
        
        # Forward Rollout
        for k = 1:(N-1)
            t = prob.times[k]
            dt = prob.times[k+1] - prob.times[k]
            dx = state_error(model, Xbar[k],X[k])
            Ubar[k] = U[k]-α*d[k]-K[k]*dx;
            Xbar[k+1] = discrete_dynamics(model,Xbar[k],Ubar[k],t,dt);
            # TODO: apply the local feedback control on the error state
        end
        
        # Calculate the new cost
        Jn = cost(prob.obj, Xbar, Ubar)

        # Check Armijo condition
        if Jn <= J - 1e-2*α*ΔJ
            break
        else
            # Backtrack
            α *= 0.5  
        end
        if i == max_iters 
            α = 0
        end
    end
    
    # Accept direction
    for k = 1:N-1
        X[k] = Xbar[k]
        U[k] = Ubar[k]
    end
    X[N] = Xbar[N]
    
    return Jn, α
end

forwardpass!

In [14]:
"""
    solve_ilqr(prob, X, U; kwargs...)

Solve the trajectory optimization problem specified by `prob` using iterative LQR.
Returns the optimized state and control trajectories, as well as the local control gains,
`K` and `d`.

Should return the optimized state and control trajectories `X` and `U`, and the 
list of feedback gains `K` and cost-to-go hessians `P`.
"""
function solve_ilqr(prob::iLQRProblem{n,m}, X0, U0; 
        iters=100,     # max iterations
        ls_iters=10,   # max line search iterations
        reg_min=1e-6,  # minimum regularizatio for the backwardpass
        verbose=0,     # print verbosity
        eps=1e-5,      # termination tolerance
        eps_ddp=eps    # tolerance to switch to ddp
    ) where {n,m}
    t_start = time_ns()
    Nx,Nu,Nt = size(prob)

    # Initialization
    # NOTE: some of these sizes have changed!
    N = prob.N
    p = [zeros(n-1) for k = 1:N]              # ctg gradient
    P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
    d = [zeros(m) for k = 1:N-1]              # feedforward gains
    K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
    Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
    Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory
    ΔJ = 0.0

    # Don't modify the trajectories that are passed in
    X = deepcopy(X0)
    U = deepcopy(U0)

    # Initial cost
    J = cost(prob.obj, X, U)
    
    # Initialize parameters
    Jn = Inf
    iter = 0
    tol = 1.0
    β = reg_min
    while tol > eps 
        iter += 1
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=β)

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=ls_iters)

        if α === zero(α) 
            β = max(β*10, 1.0)
            # β *= 10 
        else 
            β = max(β/2, reg_min)
        end

        # Update parameters
        tol = maximum(norm.(d, Inf))
        β = max(0.9*β, reg_min)

        # Output
        if verbose > 0
            @printf("Iter: %3d, Cost: % 6.2f → % 6.2f (% 7.2e), res: % .2e, β= %.2e, α = %.3f, ΔJ = %.3e\n",
                iter, J, Jn, J-Jn, tol, β, α, ΔJ
            )
        end
        J = Jn

        if iter >= iters
            @warn "Reached max iterations"
            break
        end

    end
    println("Total Time: ", (time_ns() - t_start)*1e-6, " ms")
    return X,U,K,P
end

solve_ilqr

In [15]:
using RobotDynamics: RBState
function YakProblems(;
        N = 101,
        vecstate=false,
        scenario=:barrellroll, 
        heading=0.0,  # deg
        Qpos=1.0,
        kwargs...
    )
    model = YakPlane(UnitQuaternion)

    n,m = size(model)
    ip,iq,iv,iw = state_parts(model)

    # Discretization
    tf = 1.25
    if scenario == :fullloop 
        # Double the time, keeping the same discretization
        tf *= 2
        N = (N-1)*2 + 1
    end
    dt = tf/(N-1)

    # Forward Velocity 
    vel = 5.0

    if scenario ∈ (:halfloop, :fullloop) 
        ey = @SVector [0,1,0.]

        # Heading
        dq = expm(SA[0,0,1]*deg2rad(heading))

        # Initial state
        p0 = MRP(0.997156, 0., 0.075366) # initial orientation (level flight)
        x0 = RD.build_state(model, [-3,0,1.5], p0, [vel,0,0], [0,0,0])

        # Climb
        pm = expm(SA[1,0,0]*deg2rad(180))*expm(SA[0,1,0]*deg2rad(90))
        xm = RD.build_state(model, [0,0,3.], pm, pm * [vel,0,0.], [0,0,0])

        # Top of loop
        pf = MRP(0., -0.0366076, 0.) * dq # final orientation (upside down)
        xf = RD.build_state(model, dq*[3,0,6.], pf, pf * [vel,0,0.], [0,0,0])
        pf2 = RotZ(deg2rad(heading-180))

        # Dive
        xm2 = RD.build_state(model, [-3,3,4.], pm * RotY(pi), [0,0,-vel], [0,0,0])

        # Terminal state
        xf2 = RD.build_state(model, [0,4,1.5], p0, [vel,0,0], [0,0,0])

        t_flat = 5 / (xf[2] - xf[1])
        N_flat = Int(round(t_flat/dt))

        # Xref trajectory
        x̄0 = RBState(model, x0)
        x̄m = RBState(model, xm)
        x̄m2 = RBState(model, xm2)
        x̄f = RBState(model, xf)
        x̄f2 = RBState(model, xf2)
        Xref = map(1:N) do k
            t = (k-1)/(N-1)
            Nmid = N ÷ 4
            if scenario == :fullloop
                if k < Nmid
                    x1 = x̄0
                    x2 = x̄m
                    t = (k-1)/Nmid
                elseif k < 2Nmid
                    t = (k-Nmid)/Nmid
                    x1 = x̄m
                    x2 = x̄f 
                elseif k < 3Nmid
                    t = (k-2Nmid)/Nmid
                    x1 = x̄f
                    x2 = x̄m2
                else
                    t = (k-3Nmid)/Nmid
                    x1 = x̄m2
                    x2 = x̄f2
                end
            else
                if k < 2Nmid
                    t = (k-1)/2Nmid
                    x1 = x̄0
                    x2 = x̄m
                else
                    t = (k-2Nmid)/(2Nmid+1)
                    x1 = x̄m
                    x2 = x̄f
                end
            end
            RBState(
                x1.r + (x2.r - x1.r)*t,
                slerp(x1.q, x2.q, t),
                x1.v + (x2.v - x1.v)*t,
                SA[0,pi/1.25,0]
            )
        end
    else
        throw(ArgumentError("$scenario isn't a known scenario"))
    end

    # Get trim condition
    utrim = get_trim(model, x0, fill(124, 4))

    # Objective
    Qf_diag = RD.fill_state(model, 10, 500*0, 100, 100.)
    Q_diag = RD.fill_state(model, Qpos*0.1, 0.1*0, 0.1, 1.1)
    R = Diagonal(@SVector fill(1e-3,4))
    costs = map(1:N-1) do k
        RigidBodyCost(
            Diagonal(Q_diag[ip])*dt,
            10.0*dt, 
            Diagonal(Q_diag[iv])*dt,
            Diagonal(Q_diag[iw])*dt,
            R*dt,
            Xref[k][ip],
            Xref[k][iq],
            Xref[k][iv],
            Xref[k][iw],
            SVector{4,Float64}(utrim)
        )
    end
    costterm = RigidBodyCost(
            Diagonal(Qf_diag[ip]),
            200.0, 
            Diagonal(Qf_diag[iv]),
            Diagonal(Qf_diag[iw]),
            R*0,
            Xref[N][ip],
            Xref[N][iq],
            Xref[N][iv],
            Xref[N][iw],
            SVector{4,Float64}(utrim)
        )
    push!(costs, costterm)

    # Build problem
    prob = iLQRProblem(model, costs, tf, x0)

    return prob, Xref, utrim
end

YakProblems (generic function with 1 method)

## Part (e): Solve for half of the loop (2 pts)
It turns out the maneuver we're trying to execute is a little tricky to get with a simple trim condition initializer (remember that these problems have lots of local minima). To get the desired behavior, we're going to first solve a simpler problem that will give us a good initialization for the problem we actually care about. We're basically going to solve for half of the loop first, and then ask it to get the rest of the loop.

We've taken care of the reference trajectories for this problem, but we encourage you to look at the code and at the trajectories in the visualizer.

#### Visualizer

In [16]:
if !isautograder
    vis = Visualizer()
    TrajOptPlots.set_mesh!(vis, YakPlane(UnitQuaternion), color = RGBA(0, 0, 1, 0.5))
    render(vis)  # this may take a while, it's a detailed model
end

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8703
└ @ MeshCat /Users/Augusto/.julia/packages/MeshCat/GlCMx/src/visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set{Any}(), ip"127.0.0.1", 8703))

In [17]:
# Generate the problem, and visualize the reference trajectory
prob_half, Xref_half = YakProblems(costfun=:QuatLQR, scenario=:halfloop, heading=130)
isautograder || visualize!(vis, prob_half.model, prob_half.tf, Xref_half)

In [18]:
# TASK: Generate the initial guess, using trim conditions (2 pts)
#  TIP: calculate the trim controls using get_trim
#  TIP: write a method to simulate the dynamics forward

U0 = [get_trim(prob_half.model, prob_half.x0, fill(124.0,4)) for k = 1:prob_half.N-1]  # TODO: Update this!
X0 = [SVector(prob_half.x0) for k = 1:prob_half.N]      # remember this needs to be dynamically feasible!
for k = 1:prob_half.N-1
    t = prob_half.times[k]
    dt = prob_half.times[k+1] - prob_half.times[k]
    X0[k+1] = discrete_dynamics(prob_half.model,X0[k],U0[k],t,dt);
end

isautograder || visualize!(vis, prob_half.model, prob_half.tf, X0)

In [19]:
# Solve for the first half of the trajectory
# TIP: It should take about 17 iterations
# TIP: Final cost should be about 6.2

Xhalf, Uhalf = solve_ilqr(prob_half, X0, U0, verbose=1, eps=1e-4, reg_min=1e-6)
isautograder || visualize!(vis, prob_half.model, prob_half.tf, Xhalf)

Iter:   1, Cost:  4878.39 →  4629.75 ( 2.49e+02), res:  5.81e+03, β= 1.00e-06, α = 0.062, ΔJ = 9.582e+03
Iter:   2, Cost:  4629.75 →  4310.82 ( 3.19e+02), res:  7.38e+03, β= 1.00e-06, α = 0.031, ΔJ = 8.886e+03
Iter:   3, Cost:  4310.82 →  3713.14 ( 5.98e+02), res:  8.28e+03, β= 1.00e-06, α = 0.062, ΔJ = 8.273e+03
Iter:   4, Cost:  3713.14 →  2720.21 ( 9.93e+02), res:  6.50e+03, β= 1.00e-06, α = 0.125, ΔJ = 7.442e+03


Iter:   5, Cost:  2720.21 →  1120.83 ( 1.60e+03), res:  5.00e+03, β= 1.00e-06, α = 0.500, ΔJ = 5.356e+03
Iter:   6, Cost:  1120.83 →  458.44 ( 6.62e+02), res:  2.32e+03, β= 1.00e-06, α = 0.500, ΔJ = 2.172e+03


Iter:   7, Cost:  458.44 →  178.87 ( 2.80e+02), res:  1.23e+03, β= 1.00e-06, α = 1.000, ΔJ = 8.885e+02
Iter:   8, Cost:  178.87 →  91.84 ( 8.70e+01), res:  6.03e+02, β= 1.00e-06, α = 0.500, ΔJ = 3.431e+02
Iter:   9, Cost:  91.84 →  58.80 ( 3.30e+01), res:  3.49e+02, β= 1.00e-06, α = 1.000, ΔJ = 1.700e+02
Iter:  10, Cost:  58.80 →  49.55 ( 9.25e+00), res:  1.48e+03, β= 1.00e-06, α = 1.000, ΔJ = 1.009e+02


Iter:  11, Cost:  49.55 →   7.97 ( 4.16e+01), res:  1.53e+02, β= 1.00e-06, α = 1.000, ΔJ = 8.660e+01
Iter:  12, Cost:   7.97 →   6.42 ( 1.54e+00), res:  3.13e+01, β= 1.00e-06, α = 1.000, ΔJ = 3.541e+00
Iter:  13, Cost:   6.42 →   6.20 ( 2.26e-01), res:  3.41e+00, β= 1.00e-06, α = 1.000, ΔJ = 4.537e-01
Iter:  14, Cost:   6.20 →   6.20 ( 1.43e-04), res:  1.16e+00, β= 1.00e-06, α = 1.000, ΔJ = 9.501e-04
Iter:  15, Cost:   6.20 →   6.20 ( 4.61e-05), res:  9.61e-01, β= 1.00e-06, α = 1.000, ΔJ = 5.523e-04
Iter:  16, Cost:   6.20 →   6.20 ( 3.53e-05), res:  8.00e-01, β= 1.00e-06, α = 1.000, ΔJ = 3.757e-04
Iter:  17, Cost:   6.20 →   6.20 ( 1.98e-05), res:  6.67e-01, β= 1.00e-06, α = 1.000, ΔJ = 2.575e-04
Iter:  18, Cost:   6.20 →   6.20 ( 1.70e-05), res:  5.57e-01, β= 1.00e-06, α = 1.000, ΔJ = 1.779e-04
Iter:  19, Cost:   6.20 →   6.20 ( 8.51e-06), res:  4.65e-01, β= 1.00e-06, α = 1.000, ΔJ = 1.234e-04
Iter:  20, Cost:   6.20 →   6.20 ( 8.69e-06), res:  3.88e-01, β= 1.00e-06, α = 1.000, ΔJ = 

Iter:  25, Cost:   6.20 →   6.20 ( 1.79e-07), res:  1.60e-01, β= 1.00e-06, α = 1.000, ΔJ = 1.621e-05
Iter:  26, Cost:   6.20 →   6.20 ( 1.31e-06), res:  1.50e-01, β= 1.00e-06, α = 1.000, ΔJ = 1.265e-05
Iter:  27, Cost:   6.20 →   6.20 ( 2.42e-06), res:  1.68e-01, β= 1.00e-06, α = 0.500, ΔJ = 1.069e-05
Iter:  28, Cost:   6.20 →   6.20 ( 4.59e-09), res:  1.58e-02, β= 1.00e-06, α = 0.250, ΔJ = 7.944e-08


Iter:  29, Cost:   6.20 →   6.20 (-3.72e-12), res:  7.22e-03, β= 9.00e-01, α = 0.000, ΔJ = 2.020e-08
Iter:  30, Cost:   6.20 →   6.20 ( 1.15e-14), res:  3.99e-07, β= 4.05e-01, α = 0.031, ΔJ = 1.934e-12
Total Time: 1943.286478 ms


In [20]:
@testset "Part d" begin                                                                 # POINTS = 12
    let prob = prob_half
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        Xbar = [@SVector zeros(n) for k = 1:N]    # line search trajectory
        Ubar = [@SVector zeros(m) for k = 1:N-1]  # line search trajectory

        X = deepcopy(X0)
        U = deepcopy(U0)

        # Initial cost
        J = cost(prob.obj, X, U)
        @test J ≈ load(resfile, "J0")                                                   # POINTS = 1
        
        # Backward Pass
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1e-8)
        @test ΔJ ≈ load(resfile, "dJ") rtol=1e-3                                        # POINTS = 5
        @test norm(d,Inf) ≈ load(resfile, "d0") rtol=1e-3                               # POINTS = 2

        # Forward Pass
        Jn, α = forwardpass!(prob, X, U, K, d, ΔJ, J, Xbar, Ubar, max_iters=10)
        @test Jn < J                                                                    # POINTS = 2
        @test (J-Jn) / ΔJ ≈ 0.025 atol=1e-2                                             # POINTS = 1
        @test Jn ≈ load(resfile, "Jn") rtol=1e-1                                        # POINTS = 1
    end
    
end;

Test Summary: | Pass  Total
Part d        |    6      6


In [21]:
@testset "Part e" begin                                                                 # POINTS = 2
    model = prob_half.model
    @test norm(RBState(model,Xhalf[end]) ⊖ RBState(model,Xref_half[end])) < 0.5
    @test cost(prob_half.obj, Xhalf, Uhalf) < 6.5
    let prob = prob_half, X=Xhalf, U=Uhalf
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1e-6)
        @test ΔJ < 1e-4                                                                  # POINTS = 1
        @test norm(d) < 0.1                                                              # POINTS = 1
    end
end;

Test Summary: | Pass  Total
Part e        |    4      4


## Part (f): Complete the Loop (3 pts)
Use the result of your previous solve to generate a good initial guess for the full maneuver. The new trajectory is twice as long as the previous one.

**TIP**: Reverse the control sequence for the second half of the trajectory.

In [22]:
# Generate the new problem and reference trajectory
prob, Xref_full = YakProblems(costfun=:QuatLQR, scenario=:fullloop, heading=130, Qpos=100)
isautograder || render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), UInt8[0x83, 0xa7, 0x6f, 0x70, 0x74, 0x69, 0x6f, 0x6e, 0x73, 0x82  …  0x5f, 0x61, 0x6e, 0x69, 0x6d, 0x61, 0x74, 0x69, 0x6f, 0x6e], Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("robot" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geom" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6f, 0x62, 0x6f, 0x74, 0x2f, 0x67, 0x65, 0x6f, 0x6d], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), Set(Any[WebSocket(server, CONNECTED)]), ip"127.0.0.1", 8703))

In [23]:
# Visualize the reference trajectory
isautograder || visualize!(vis, prob.model, prob.tf, Xref_full)

In [24]:
# TASK: Design a good initialization, using result for the previous problem (2 pts)
U0 = deepcopy(Uhalf)  # TODO: update this
U0_2 = reverse(U0)
U0 =[U0;U0_2]
X0 = deepcopy(Xref_full)
for k = 1:prob.N-1
    t = prob.times[k]
    dt = prob.times[k+1] - prob.times[k]
    X0[k+1] = discrete_dynamics(prob.model,X0[k],U0[k],t,dt);
end


isautograder || visualize!(vis, prob.model, prob.tf, X0)

In [25]:
# Solve for the full loop
# TIP: This will probably take between 100-200 iterations
# TIP: Final cost should be about 14
# TIP: the regularization parameter has already been tuned, but feel free to change it if needed
# NOTE: We've relaxed the optimality condition for this solve, to make it easier

Xfull, Ufull = solve_ilqr(prob, X0, U0, verbose=1, eps=1e-2, reg_min=1e-8, iters=500)
isautograder || visualize!(vis, prob.model, prob.tf, Xfull)

Iter:   1, Cost:  10040.73 →  5329.89 ( 4.71e+03), res:  1.01e+04, β= 1.00e-08, α = 0.250, ΔJ = 2.002e+04
Iter:   2, Cost:  5329.89 →  4043.69 ( 1.29e+03), res:  7.23e+03, β= 1.00e-08, α = 0.125, ΔJ = 1.061e+04
Iter:   3, Cost:  4043.69 →  3271.57 ( 7.72e+02), res:  6.06e+03, β= 1.00e-08, α = 0.062, ΔJ = 8.040e+03
Iter:   4, Cost:  3271.57 →  2850.76 ( 4.21e+02), res:  4.28e+03, β= 1.00e-08, α = 0.062, ΔJ = 6.509e+03
Iter:   5, Cost:  2850.76 →  2418.66 ( 4.32e+02), res:  4.71e+03, β= 1.00e-08, α = 0.062, ΔJ = 5.667e+03
Iter:   6, Cost:  2418.66 →  1667.12 ( 7.52e+02), res:  4.90e+03, β= 1.00e-08, α = 0.125, ΔJ = 4.802e+03
Iter:   7, Cost:  1667.12 →  1139.96 ( 5.27e+02), res:  3.49e+03, β= 1.00e-08, α = 0.125, ΔJ = 3.296e+03
Iter:   8, Cost:  1139.96 →  866.59 ( 2.73e+02), res:  2.27e+03, β= 1.00e-08, α = 0.125, ΔJ = 2.246e+03


Iter:   9, Cost:  866.59 →  667.93 ( 1.99e+02), res:  2.81e+03, β= 1.00e-08, α = 0.125, ΔJ = 1.694e+03
Iter:  10, Cost:  667.93 →  522.25 ( 1.46e+02), res:  1.38e+03, β= 1.00e-08, α = 0.125, ΔJ = 1.300e+03
Iter:  11, Cost:  522.25 →  408.64 ( 1.14e+02), res:  9.73e+02, β= 1.00e-08, α = 0.125, ΔJ = 1.008e+03
Iter:  12, Cost:  408.64 →  351.75 ( 5.69e+01), res:  1.47e+03, β= 1.00e-08, α = 0.250, ΔJ = 7.856e+02


Iter:  13, Cost:  351.75 →  251.67 ( 1.00e+02), res:  1.45e+03, β= 1.00e-08, α = 0.250, ΔJ = 6.723e+02
Iter:  14, Cost:  251.67 →  223.87 ( 2.78e+01), res:  7.40e+02, β= 1.00e-08, α = 0.125, ΔJ = 4.762e+02
Iter:  15, Cost:  223.87 →  213.42 ( 1.04e+01), res:  6.97e+02, β= 1.00e-08, α = 0.125, ΔJ = 4.205e+02
Iter:  16, Cost:  213.42 →  202.95 ( 1.05e+01), res:  7.41e+02, β= 1.00e-08, α = 0.031, ΔJ = 3.990e+02
Iter:  17, Cost:  202.95 →  201.29 ( 1.66e+00), res:  7.81e+02, β= 1.00e-08, α = 0.016, ΔJ = 3.774e+02
Iter:  18, Cost:  201.29 →  192.97 ( 8.32e+00), res:  8.42e+02, β= 1.00e-08, α = 0.500, ΔJ = 3.731e+02
Iter:  19, Cost:  192.97 →  127.74 ( 6.52e+01), res:  4.32e+02, β= 1.00e-08, α = 0.250, ΔJ = 3.601e+02
Iter:  20, Cost:  127.74 →  125.82 ( 1.92e+00), res:  2.90e+02, β= 1.00e-08, α = 0.125, ΔJ = 2.292e+02


Iter:  21, Cost:  125.82 →  121.30 ( 4.52e+00), res:  2.01e+02, β= 1.00e-08, α = 0.062, ΔJ = 2.245e+02
Iter:  22, Cost:  121.30 →  117.15 ( 4.15e+00), res:  2.60e+02, β= 1.00e-08, α = 0.031, ΔJ = 2.151e+02
Iter:  23, Cost:  117.15 →  116.52 ( 6.30e-01), res:  2.68e+02, β= 1.00e-08, α = 0.031, ΔJ = 2.067e+02
Iter:  24, Cost:  116.52 →  114.47 ( 2.05e+00), res:  1.96e+02, β= 1.00e-08, α = 0.031, ΔJ = 2.072e+02
Iter:  25, Cost:  114.47 →  111.41 ( 3.06e+00), res:  2.38e+02, β= 1.00e-08, α = 0.031, ΔJ = 2.015e+02
Iter:  26, Cost:  111.41 →  110.16 ( 1.24e+00), res:  1.82e+02, β= 1.00e-08, α = 0.062, ΔJ = 1.958e+02


Iter:  27, Cost:  110.16 →  104.53 ( 5.64e+00), res:  2.43e+02, β= 1.00e-08, α = 0.031, ΔJ = 1.944e+02
Iter:  28, Cost:  104.53 →  103.84 ( 6.91e-01), res:  1.80e+02, β= 1.00e-08, α = 0.016, ΔJ = 1.832e+02


...

Iter: 461, Cost:  13.73 →  13.73 ( 1.24e-03), res:  4.97e-01, β= 3.36e-03, α = 1.000, ΔJ = 8.871e-03
Iter: 462, Cost:  13.73 →  13.73 ( 6.40e-04), res:  2.44e+00, β= 1.51e-03, α = 0.062, ΔJ = 5.949e-02
Iter: 463, Cost:  13.73 →  13.72 ( 2.53e-03), res:  2.34e+00, β= 6.81e-04, α = 0.250, ΔJ = 3.733e-02
Iter: 464, Cost:  13.72 →  13.72 ( 1.31e-03), res:  6.90e+00, β= 3.06e-04, α = 0.008, ΔJ = 2.289e-01
Iter: 465, Cost:  13.72 →  13.72 ( 3.29e-04), res:  1.18e+01, β= 1.38e-04, α = 0.016, ΔJ = 3.221e-01
Iter: 466, Cost:  13.72 →  13.72 ( 1.57e-04), res:  1.24e+01, β= 6.21e-05, α = 0.002, ΔJ = 3.541e-01
Iter: 467, Cost:  13.72 →  13.72 ( 3.92e-03), res:  2.50e+01, β= 2.79e-05, α = 0.016, ΔJ = 5.548e-01
Iter: 468, Cost:  13.72 →  13.72 ( 8.71e-06), res:  3.29e+01, β= 9.00e-01, α = 0.000, ΔJ = 1.253e+00


Iter: 469, Cost:  13.72 →  13.71 ( 6.71e-03), res:  4.70e-02, β= 4.05e-01, α = 1.000, ΔJ = 7.181e-03
Iter: 470, Cost:  13.71 →  13.71 ( 5.04e-03), res:  5.37e-02, β= 1.82e-01, α = 1.000, ΔJ = 5.852e-03
Iter: 471, Cost:  13.71 →  13.70 ( 2.69e-03), res:  6.21e-02, β= 8.20e-02, α = 1.000, ΔJ = 3.235e-03
Iter: 472, Cost:  13.70 →  13.70 ( 1.81e-03), res:  5.99e-02, β= 3.69e-02, α = 1.000, ΔJ = 2.324e-03
Iter: 473, Cost:  13.70 →  13.70 ( 6.42e-04), res:  1.17e-01, β= 1.66e-02, α = 1.000, ΔJ = 3.015e-03
Iter: 474, Cost:  13.70 →  13.70 ( 1.98e-04), res:  2.30e-01, β= 7.47e-03, α = 1.000, ΔJ = 7.652e-03


Iter: 475, Cost:  13.70 →  13.70 ( 2.44e-03), res:  8.31e-01, β= 3.36e-03, α = 0.125, ΔJ = 2.240e-02
Iter: 476, Cost:  13.70 →  13.70 ( 5.82e-04), res:  1.10e+00, β= 1.51e-03, α = 0.031, ΔJ = 2.371e-02
Iter: 477, Cost:  13.70 →  13.70 (-5.07e-03), res:  2.25e+00, β= 9.00e-01, α = 0.000, ΔJ = 3.799e-02
Iter: 478, Cost:  13.70 →  13.70 ( 1.96e-04), res:  1.98e-02, β= 4.05e-01, α = 0.125, ΔJ = 1.019e-03
Iter: 479, Cost:  13.70 →  13.70 ( 3.03e-05), res:  3.71e-02, β= 1.82e-01, α = 0.016, ΔJ = 1.395e-03
Iter: 480, Cost:  13.70 →  13.70 ( 1.57e-03), res:  6.54e-02, β= 8.20e-02, α = 1.000, ΔJ = 2.074e-03
Iter: 481, Cost:  13.70 →  13.70 ( 4.22e-04), res:  8.30e-02, β= 3.69e-02, α = 0.500, ΔJ = 7.406e-03
Iter: 482, Cost:  13.70 →  13.70 ( 5.32e-03), res:  1.07e-01, β= 1.66e-02, α = 1.000, ΔJ = 2.479e-03


Iter: 483, Cost:  13.70 →  13.69 ( 1.28e-03), res:  4.26e-01, β= 7.47e-03, α = 0.250, ΔJ = 9.366e-03
Iter: 484, Cost:  13.69 →  13.69 ( 6.00e-04), res:  4.47e-01, β= 3.36e-03, α = 0.062, ΔJ = 8.433e-03
Iter: 485, Cost:  13.69 →  13.69 ( 2.76e-04), res:  9.44e-01, β= 1.51e-03, α = 0.016, ΔJ = 1.649e-02
Iter: 486, Cost:  13.69 →  13.69 ( 7.00e-05), res:  1.96e+00, β= 6.81e-04, α = 0.002, ΔJ = 3.459e-02
Iter: 487, Cost:  13.69 →  13.70 (-4.99e-03), res:  3.90e+00, β= 9.00e-01, α = 0.000, ΔJ = 7.340e-02
Iter: 488, Cost:  13.70 →  13.70 ( 8.01e-04), res:  7.90e-03, β= 4.05e-01, α = 1.000, ΔJ = 8.767e-04


Total Time: 7495.491043 ms


In [26]:
@testset "Part f" begin                                                                     # POINTS = 3
    @test norm(RBState(prob.model,Xfull[end]) ⊖ RBState(prob.model,Xref_full[end])) < 0.5
    @test cost(prob.obj, Xfull, Ufull) < 14.5
    let prob = prob, X=Xfull, U=Ufull
        n,m,N = size(prob)
        p = [zeros(n-1) for k = 1:N]              # ctg gradient
        P = [zeros(n-1,n-1) for k = 1:N]          # ctg hessian
        d = [zeros(m) for k = 1:N-1]              # feedforward gains
        K = [zeros(m,n-1) for k = 1:N-1]          # feedback gains
        ΔJ, = backwardpass!(prob, P, p, K, d, X, U, β=1.0)
        @test ΔJ < 1.0                                                                      # POINTS = 1
        @test norm(d) < 1.0                                                                 # POINTS = 2
    end
end;

Test Summary: | Pass  Total
Part f        |    4      4
